In [1]:
import os
import regex
import json
import datetime, pytz
import speech_recognition as sr

from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import ToolMessage, HumanMessage, SystemMessage, AIMessage
from langchain.tools import tool, StructuredTool

from db_search import get_info
from text_to_speech import Text2Speech

In [2]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="suFEUjmnfvT2AQSF5b6MQJ9NyZwDLsqG"
)

In [3]:
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("_"*100)
        try:
            audio = recognizer.listen(source, timeout=3000)
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            return None
        except sr.WaitTimeoutError:
            return None

In [4]:
@tool

def get_user_data() -> dict:
    """
    Returns all information about the customer's loan.
    """
    key = int(phone)
    user_data = get_info(key)
    return user_data

In [5]:
@tool
def current_date_time() -> dict:
    '''
    Returns the current server date and time in JSON format.
    '''
    now = datetime.datetime.now()
    ist_timezone = pytz.timezone('Asia/Kolkata')
    dt_ist = now.astimezone(ist_timezone)
    time = dict()
    time['day'] = dt_ist.strftime('%A')
    time['month'] = dt_ist.strftime('%B')
    time['date'] = dt_ist.strftime('%Y-%m-%d')
    time['time'] = dt_ist.strftime('%H:%M')
    return time

In [6]:
llm_with_tools = llm.bind_tools([get_user_data, current_date_time])

tool_mapping = {
    "get_user_data" : get_user_data,
    "current_date_time" : current_date_time
}

In [8]:
#Refresh convo.

template = """You are an intelligent virtual financial agent helping our customer {first_name} {last_name}.
            Your role is to help manage the customer's loan repayment and answer their financial questions in a clear and precise way. 

            Instructions:
            1. Use precise financial language and ensure clear, accurate information.
            2. If the user is willing to pay the loan then please provide this link '''https://paymentUSER1UDN.com'''. Do not send the link until user requests or user wants to pay the loan.
            3. If the customer is struggling, provide options like grace periods, payment restructuring, or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
            4. Keep responses short and to the point.
            5. Ensure confidentiality and remind the customer to keep their payment details secure.
            6. You can only extend the last loan repayment date by a maximum of 10 days if user requests for grace periods or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
            7. If the question cannot be answered using the information provided, reply with "Sorry, but I am unable to answer this query". 
            
"""

chat_history = []

In [9]:
speaker = Text2Speech()

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


B:\Software\Anaconda\envs\voicecloning5\lib\site-packages\TTS\tts\layers\xtts\xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(spe

In [10]:
template_messages = [
    SystemMessage(content=template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}")
    #HumanMessage(content="{query}")
]

prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [11]:
phone = 9804604602

In [12]:
user_data = get_info(phone)

user_info = {
    "first_name": user_data['first_name'],
    "last_name": user_data['last_name']
}

In [13]:
formatted_template = template.format(**user_info)
template_messages[0] = SystemMessage(content=formatted_template)
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [14]:
def chat(text:str) -> str:
    messages = prompt_template.format_messages(
        chat_history=chat_history,
        query = text
    )
    chat_history.append(HumanMessage(text))
    
    response = llm_with_tools.invoke(messages)
    chat_history.append(response)
    
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool = tool_mapping[tool_call['name']]
            output = tool.invoke(tool_call['args'])
            chat_history.append(ToolMessage(str(output), tool_call_id=tool_call['id']))
            ai_says = llm_with_tools.invoke(chat_history)
        chat_history.append(ai_says)
    else:
        ai_says = response

    return ai_says.content

In [20]:
welcome = AIMessage(content=f"Hello {user_info['first_name']}, I am here to talk about you loan repayment.")
chat_history.append(welcome)
print('Bot: ',welcome.content)
speaker.speak(welcome.content)

while True:
    print("Listening.....")
    query = recognize_speech()
    if query is not None:
        if query.lower() == 'okay thank you' or 'ok thank you':
            res = chat(query)
            speaker.speak(res)
            break
        else:
            res = chat(query)
            cleaned_text = regex.sub(r'[\/@|!?]', '', res)
            print('Bot: ',cleaned_text)
            speaker.speak(cleaned_text)
            

Bot:  Hello Diya, I am here to talk about you loan repayment.
 > Text splitted to sentences.
['Hello Diya, I am here to talk about you loan repayment.']
 > Processing time: 3.3000733852386475
 > Real-time factor: 0.5884219994866103
Listening.....
____________________________________________________________________________________________________
You said: hello
Bot:  Hello Diya,

How can I assist you today with your loan management
 > Text splitted to sentences.
['Hello Diya,', 'How can I assist you today with your loan management']
 > Processing time: 1.7292234897613525
 > Real-time factor: 0.23637037510686015
Listening.....
____________________________________________________________________________________________________
You said: tell me my loan amount
Bot:  Your loan amount is 36100.6 INR.
 > Text splitted to sentences.
['Your loan amount is 36100.6 INR.']
 > Processing time: 1.5481913089752197
 > Real-time factor: 0.26195225876997846
Listening.....
______________________________

KeyboardInterrupt: 

In [ ]:
while True:
    query = input('You Said:')
    if query is not None:
        if query.lower() == 'thanks':
            res = chat(query)
            speaker.speak(res)
            break
        else:
            res = chat(query)
            cleaned_text = regex.sub(r'[\/@|!?]', '', res)
            print('Bot: ',cleaned_text)
            speaker.speak(cleaned_text)
            

In [26]:
transcript = [convo.content for convo in chat_history]
transcript

['Hello',
 "Hello Diya! I'm here to help with your loan management needs. How can I assist you today?",
 'thanks',
 "You're welcome! If you have any questions or need assistance with your loan, feel free to ask.",
 'Hello',
 'Hello Diya! How can I assist you with your loan today?',
 'thanks',
 "You're welcome, Diya! If you have more questions or need further assistance, just let me know.",
 'Hello',
 'Hello Diya! How can I assist you with your loan today?',
 'What amount do I have pending',
 '',
 "{'first_name': 'Diya', 'last_name': 'Sharma', 'phone_no': 9804604602, 'gender': 'Female', 'income_in_inr': 380418.9, 'credit_score': 808, 'loan_type': 'Consumer Durable Loan', 'loan_amount': 36100.6, 'interest_rate': 12.4, 'process_fee': 361.0, 'installment': 6236.2, 'start_date': '2024-05-09', 'tenure': 6, 'balance_to_pay': 25866.4, 'payment_mode': 'Debit Card', 'late_payment': 0, 'last_date': '2024-08-03'}"]